In [1]:
import sys

import json
from nornir import InitNornir
from nornir.core.filter import F

from nornir.core.plugins.inventory import InventoryPluginRegister
from nornir_ansible.plugins.inventory.ansible import AnsibleInventory

from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get

from nornir_netmiko import netmiko_send_command

InventoryPluginRegister.register("inventory", AnsibleInventory)

In [2]:
def nornir_connect_and_run_command(task, plugin, action, params, platform, username, password):
  task.host.open_connection(plugin, configuration=task.nornir.config, platform=platform, username=username, password=password)
  r = task.run(
    task=action,
    command_string=params[0]
  )
  task.host.close_connection(plugin)

In [3]:
def nornir_connect_and_run_getters(task, plugin, action, params, platform, username, password):
  task.host.open_connection(plugin, configuration=task.nornir.config, platform=platform, username=username, password=password)
  r = task.run(
    task=action,
    getters=params
  )
  task.host.close_connection(plugin)

In [8]:
topology="nanog_hackathon"

In [18]:
# Initialize Nornir object with Containerlab ansible inventory
nrinit = InitNornir(
  runner={
      "plugin": "threaded",
      "options": {
          "num_workers": 10,
      },
  },
  inventory={
      "plugin": "AnsibleInventory",
      "options": {
          "hostsfile": f"../clab/clab-{topology}/ansible-inventory.yml"
      },
  },
)

kinds_platforms = {
    'ceos':     'eos',
#    'crpd':     'junos', 
    'linux':     'linux', 
#    'vr-veos':  'eos',
#    'vr-vmx':   'junos', 
#    'vr-xrv9k': 'iosxr',
}

kinds_credentials = {
    'linux':    {"username": "root",  "password": "root"},
    'ceos':     {"username": "admin", "password": "admin"},
    'crpd':     {"username": "root",  "password": "clab123"},
}

kinds_tasks = {
    'linux':     nornir_connect_and_run_command,
    'ceos':      nornir_connect_and_run_getters,
    'crpd':      nornir_connect_and_run_command,
}

kinds_plugins = {
    'linux':     "netmiko",
    'ceos':      "napalm",
    'crpd':      "netmiko",
}

kinds_actions = {
    'linux':     netmiko_send_command,
    'ceos':      napalm_get,
    'crpd':      netmiko_send_command,
}

kinds_params = {
    'linux':    ["ip -json address show"], # single element only
    'ceos':     ["facts", "interfaces", "lldp_neighbors"],
    'crpd':     ["ip -json address show"], # single element only
}

node_data = {
    "name": topology,
    "type": "node-data",
    "nodes": {},
}

nodes = {}
results = []
for k, v in kinds_platforms.items():
    nr = nrinit.filter(F(groups__contains=k))
    r = nr.run(
      task=kinds_tasks[k],
      plugin=kinds_plugins[k],
      action=kinds_actions[k],
      params=kinds_params[k],
      platform=v,
      username=kinds_credentials[k]["username"],
      password=kinds_credentials[k]["password"],
    )
    results.append({"kind": k, "result": r})

In [39]:
for r in results:
    kind = r["kind"]
    for k, v in r["result"].items():
        if not v[0].failed:
            n = {}
            n |= {"kind": kind}
            r = v[1].result
            if kind == "linux":
                interfaces_array = json.loads(r)
                interfaces = {}
                for i in interfaces_array:
                    if "link_index" in i:
                        interfaces |= {i["ifname"]: i}
                n |= {"interface_list": list(interfaces.keys())}
                n |= {"interfaces": interfaces}
            else:
                for block in r:
                    if block == "facts":
                        n |= r["facts"] # flatten "facts"
                    else:
                        n |= {block: r[block]}
            nodes |= {k: n}
                
        else:
            print(f"Connection failed for: {k}. Error: {v[0]}")

In [40]:
node_data["nodes"] |= nodes
print(json.dumps(node_data, indent=4))

{
    "name": "nanog_hackathon",
    "type": "node-data",
    "nodes": {
        "clab-nanog_hackathon-ceos1": {
            "kind": "ceos",
            "hostname": "ceos1",
            "fqdn": "ceos1",
            "vendor": "Arista",
            "model": "cEOSLab",
            "serial_number": "123B907673481DF3C5B4EC3F7DE0E792",
            "os_version": "4.28.0F-26633270.googleb4rel",
            "uptime": 905.2119526863098,
            "interface_list": [
                "Ethernet1",
                "Ethernet2",
                "Loopback0",
                "Management0"
            ],
            "interfaces": {
                "Management0": {
                    "is_up": true,
                    "is_enabled": true,
                    "description": "",
                    "last_flapped": 1663219652.696568,
                    "mtu": 1500,
                    "speed": 1000.0,
                    "mac_address": "00:1C:73:D9:5A:FF"
                },
                "Loopback0": {


}
